In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 98% 309M/315M [00:04<00:00, 125MB/s]
100% 315M/315M [00:04<00:00, 80.3MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'apnea-ecg-database.zip',
 'kaggle.json',
 'rri-amplitudes.ipynb',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [55:39<00:00,  6.89s/it]

Testing...
x12: 100%|██████████| 527/527 [57:11<00:00,  6.51s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import  Dense,Flatten,MaxPooling2D,Conv2D,Permute,Reshape,LSTM,BatchNormalization,Bidirectional
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D,Conv2D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd
from keras.layers import GRU
from keras.layers import add, concatenate
from keras.layers import Reshape, Lambda

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy

from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from tensorflow.keras import optimizers
#from Hyperactive.hyperactive import RandomSearchOptimizer, MHoneyBadgerAlgorithm, HoneyBadgerAlgorithm
from keras.optimizers import adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA
from tensorflow.python.keras import optimizers
from keras.optimizers import adam
#from HBA import HBA
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 31.2 MB/s eta 0:00:00


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7],position[8],position[9],position[10],position[11],position[12],position[13],position[14],position[15],position[16]
    n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(n7),int(n8),int(n9),int(n10),int(n11),int(n12),int(n13),int(n14),int(n15),int(epochs)
    #return learning_rate,n1,n2,n3,n4,n5,n6,epochs

    print(learning_rate,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15,epochs)

#def create_model(weight=1e-3):
    model=Sequential()
    model.add(Reshape((450,2,2),input_shape=(900,1,2)))
    model.add(Conv2D(n1, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight),input_shape=(900,1,2)))
    model.add(Conv2D(n2, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n3, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n6, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n7, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n8, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n9, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n10, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n11, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n12, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n13, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Permute((2,1,3)))
    model.add(Reshape((2,28*n13)))

    model.add(GRU(n14, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(n14, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,n15,epochs


    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    #x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    #x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    #x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,40,40,80,80,128,128,128,128,300,300,300,300,256,32,19,100], # Lower bound of our parameters
    'ub':[0.001,256,256,500,500,700,700,700,700,900,900,900,900,500,64,50,100], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalHBA(epoch, pop_size)
  #t1 = time.time()
  #Optimizer.fit(x_train, y_train)
 # t2 = time.time()

  #print("time: {}".format(t2-t1))

#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.001 184 125 235 362 209 469 427 246 490 600 586 317 369 42 31 100


INFO:mealpy.swarm_based.HBA.OriginalHBA:Solving 17-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ].


0.001 144 192 145 161 231 238 132 513 766 422 431 329 270 53 35 100
0.001 132 177 312 109 235 555 686 285 387 444 786 590 270 62 49 100
0.001 243 227 369 392 413 680 319 266 577 304 375 871 342 57 28 100
0.001 70 180 333 481 568 554 564 635 454 758 601 813 287 61 31 100
0.001 164 191 226 267 517 645 472 406 828 683 550 607 419 41 46 100
0.001 74 69 471 360 562 430 267 332 637 699 706 482 402 47 28 100
0.001 107 132 447 467 562 203 620 616 811 585 468 631 396 34 23 100
0.001 182 172 357 264 418 218 218 177 777 586 846 799 319 32 24 100
0.001 176 246 196 469 577 554 494 346 353 511 528 749 281 48 40 100
0.001 98 222 397 280 447 502 540 376 573 615 863 656 307 49 43 100
0.001 56 116 452 402 226 144 458 617 487 784 331 327 393 48 22 100
0.001 63 44 457 298 229 448 591 269 743 806 641 343 259 63 32 100
0.001 56 244 328 348 388 352 559 129 494 471 652 372 319 56 49 100
0.001 163 205 369 104 232 169 616 145 898 338 639 437 454 50 28 100
0.001 231 189 472 478 305 492 479 246 620 697 815 855 37

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 1, Current best: 3754.0004, Global best: 3754.0004, Runtime: 37.19550 seconds


0.001 40 175 80 191 141 197 128 498 300 300 300 314 258 32 32 100
0.001 86 166 80 111 161 146 170 479 558 300 447 309 266 42 31 100
0.001 110 117 153 137 143 259 142 471 460 300 307 498 300 44 31 100
0.001 155 118 100 133 128 147 155 451 536 313 445 300 256 41 32 100
0.001 138 97 135 109 197 203 128 279 696 300 300 300 256 42 20 100
0.001 151 86 158 197 161 205 238 405 543 300 563 300 256 52 39 100
0.001 192 153 122 212 128 172 209 450 740 311 581 300 256 40 34 100
0.001 158 129 80 317 128 184 238 700 655 403 510 300 371 34 45 100
0.001 142 141 120 189 274 198 203 477 655 366 441 393 267 46 33 100
0.001 147 207 130 287 128 208 303 412 587 300 634 429 256 33 45 100
0.001 161 186 106 206 128 232 201 483 673 371 448 300 321 52 39 100
0.001 217 190 193 202 182 155 220 435 663 300 632 300 256 38 43 100
0.001 136 174 80 207 163 128 163 445 300 300 452 331 256 32 32 100
0.001 124 148 138 211 136 174 228 536 727 300 462 327 256 38 20 100
0.001 149 218 80 163 128 148 241 548 664 300 717 300 383

...

0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 256 500 80 700 700 700 128 300 300 900 900 500 32 19 100
0.001 40 40 80 80 143 143 143 143 336 300 300 336 287 32 21 100
0.001 40 256 500 500 700 700 700 700 900 900 900 900 256 64 50 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 51 80 80 164 128 164 164 300 385 300 300 329 41 19 100
0.001 40 40 80 80 128 700 128 128 300 300 300 300 500 32 50 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 81 130 130 130 129 300 305 300 304 260 32 19 100
0.001 48 40 80 80 154 154 154 128 362 300 362 362 256 32 22 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 256 80 80 128 700 128 128 

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 99, Current best: 2359.0004, Global best: 2359.0004, Runtime: 32.37014 seconds


0.001 256 40 500 80 700 700 700 700 300 300 300 900 256 32 50 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 256 80 500 128 700 700 128 900 900 900 900 500 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 81 130 130 128 130 300 300 304 300 256 32 19 100
0.001 57 40 115 80 185 128 185 128 434 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 500 500 128 128 128 128 300 900 300 900 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 46 40 92 92 148 128 128 148 300 300 348 300 297 32 22 100
0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
0.001 40 40 80 80 128 128 128 128

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 100, Current best: 2359.0004, Global best: 2359.0004, Runtime: 34.77263 seconds


0.001 40 40 80 80 128 128 128 128 300 300 300 300 256 32 19 100
Solution: [1.00e-03 4.00e+01 4.00e+01 8.00e+01 8.00e+01 1.28e+02 1.28e+02 1.28e+02
 1.28e+02 3.00e+02 3.00e+02 3.00e+02 3.00e+02 2.56e+02 3.20e+01 1.90e+01
 1.00e+02], Fitness: (0.001, 40, 40, 80, 80, 128, 128, 128, 128, 300, 300, 300, 300, 256, 32, 19, 100)
